## This code contains help functions

In [ ]:
from typing import Callable, Any

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
import numpy as np
import scipy
from scipy.sparse import csr_matrix
from pathlib import Path
from torch.utils.data import DataLoader
export_dir = os.getcwd()
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import plot
import random
import math
import heapq
from scipy.special import expit  # Sigmoid function
import itertools
from IPython.display import Latex, display
import pickle
import warnings

# Ignore FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
torch.set_printoptions(sci_mode=False)

test_flag = 1

In [ ]:
pip install ipynb

In [ ]:
from ipynb.fs.defs.data_processing import *
from ipynb.fs.defs.models import *
from ipynb.fs.defs.training import *

Test sets:

In [ ]:
# from data processing
def test_set_gen(df_recommender_user_emb=df_user_mf, df_recommender_item_emb=df_item_mf):
  test_subset_users = random.sample(list(df_recommender_user_emb.index), k=math.floor(df_recommender_user_emb.shape[0]*0.2))
  test_subset_items = random.sample(list(df_recommender_item_emb.index), k=math.floor(df_recommender_item_emb.shape[0]*0.2))

  return test_subset_users, test_subset_items

Load models for future use

In [ ]:
# from model intialization- init test sets for users and items:
test_subset_users, test_subset_items = test_set_gen()

In [ ]:
#mf SAE
test_flag=1
autoencoder = Autoencoder(latent_dim, input_dim, activation=nn.ReLU(), tied=True, normalize = True)
train_autoencoder(interaction_embeddings, dataset_items,dataset_users[test_subset_users],input_dim=dataset_users.shape[1], latent_dim=22)

# NCF SAE
sae_model = SparseAutoencoderNCF(input_dim=20, hidden_dim=22, topk=7, tie_weights=True)

# MF
mf_recommender = MatrixFactorization(ratings_matrix, pos_idx_ex_use,neg_idx_ex_use,neg_ex_hidden, neg_ex, pos_ex_num, K=22, alpha=0.05, beta=0.01, iterations=6, pop_flag = 1)

# NCF
model = NeuralCollaborativeFiltering(num_users=6039, num_items=3706,
                                      embedding_dim=20, hidden_layers=[8, 4])

# KL loss term implementation

In [ ]:
def kl_divergence_loss(latent_activations, sparsity_target=0.05, eps=1e-6):

    # Calculate average activation of each latent unit and clamp to avoid exact 0 or 1
    rho_hat = torch.clamp(torch.mean(latent_activations, dim=0), eps, 1 - eps)
    rho = torch.tensor(sparsity_target, dtype=torch.float32, device=latent_activations.device)

    # Compute the KL divergence with eps added inside the log to ensure numerical stability
    kl_div = rho * torch.log((rho + eps) / (rho_hat + eps)) + \
             (1 - rho) * torch.log(((1 - rho) + eps) / ((1 - rho_hat) + eps))

    return torch.sum(kl_div)

In [ ]:
def LN(x: torch.Tensor, eps: float = 1e-5) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    if type(x) == np.ndarray:
      x= torch.from_numpy(x)
    mu = x.mean(dim=-1, keepdim=True)
    x = x - mu
    std = x.std(dim=-1, keepdim=True)
    x = x / (std + eps)
    return x, mu, std


def preprocess(x):
        x, mu, std = LN(x)
        return x, dict(mu=mu, std=std)

#Generating result table

In [ ]:
def table_maker_Wchoice(table_size:int, objects:list, main_data=df_movie, main_data_names=df_movie_names, pop_ranking=num_users_per_movie_sort, selected_columns:list= ['Movie ID', 'Name', 'Year', 'Genre','Pop_Percent']):
    table = pd.DataFrame(0, index=range(table_size), columns=selected_columns)


    for i in table.index:
      if 'Name' in selected_columns:
        pos_name = selected_columns.index('Name')
        table.iloc[i,pos_name] =  main_data_names.loc[objects[i]].iloc[0][0:-6]
      if 'Movie ID' in selected_columns:
        pos_id = selected_columns.index('Movie ID')
        table.iloc[i,pos_id] =  objects[i]
      if 'Year' in selected_columns:
        pos_year = selected_columns.index('Year')
        table.iloc[i,pos_year] =  df_movie_names.loc[neuron_data[i]].iloc[0,][-5:-1]
      if 'Genre' in selected_columns:
        pos_genre = selected_columns.index('Genre')
        table.iloc[i,pos_genre] = ', '.join(main_data.columns[(np.where(main_data.loc[objects[i]]==1)[0])].tolist())
      if 'Pop_Percent' in selected_columns:
        pos_pop = selected_columns.index('Pop_Percent')
        table.iloc[i,pos_pop] = np.where(pop_ranking.index == objects[i])[0][0]


    return table

In [ ]:
def table_maker(table_size:int, objects:list, main_data=df_movie, main_data_names=df_movie_names, pop_ranking=num_users_per_movie_sort):
    table = pd.DataFrame(0, index=range(table_size), columns=['movie', 'name', 'genres','pop rank'])
    for i in table.index:
      table.iloc[i,0] =  main_data.index[objects[i]]
      table.iloc[i,1] =  main_data_names.iloc[objects[i]].iloc[0]
      table.iloc[i,2] =  ', '.join(main_data.columns[(np.where(main_data.iloc[objects[i]]==1)[0])].tolist())
      table.iloc[i,3] =  np.where(pop_ranking.index == main_data.index[objects[i]])[0][0]
    return table



def table_maker_new(table_size:int, objects:list, main_data=df_movie, main_data_names=df_movie_names, pop_ranking=num_users_per_movie_sort):
    table = pd.DataFrame(0, index=range(table_size), columns=['Name', 'Movie ID', 'Genre'])
    for i in table.index:
      table.iloc[i,0] =  main_data_names.iloc[objects[i]].iloc[0]
      table.iloc[i,1] =  main_data.index[objects[i]]
      table.iloc[i,2] =  ', '.join(main_data.columns[(np.where(main_data.iloc[objects[i]]==1)[0])].tolist())

    return table

# Generating group of users w.r.t a certain concept:

In [ ]:
'''find the group'''
'''list of users that watched movies of certain genre, sorted by the number of
movies of the certain genre wrt the total number of movies ranked by the user'''
# all_genres = ['Children\'s','Drama', 'Action','Comedy', 'Adventure','Documentary',
#               'Animation', 'Crime','Fantasy','Film-Noir','Mystery', 'Horror',
#               'Musical', 'Romance','Sci-Fi', 'Thriller', 'War', 'Western']

def generate_users_test_group(group_concept, N,model_name):

  'generates group of users who have cetain dominant preferences of specific concept'

  concept_ids = np.where(df_movie[group_concept] == 1)

  group_concept_movies_name = np.array(df_movie.iloc[concept_ids]['movie_id'])
  rel_group_concept_movies_name = [movie for movie in group_concept_movies_name if movie in ratings_matrix.columns]
  part_sum_for_group_concept=ratings_matrix.loc[:,list(rel_group_concept_movies_name)].sum(axis=1)/ratings_matrix.sum(axis=1)
  argmax_user = part_sum_for_group_concept.iloc[model_name.test_subset_users_ind].nlargest(N).index # N users watched the biggest amount of 'genre' movies vs the total num of movies

  # users watched most of group_concept movies, the number of group_concept movies
  # they watched, and the number of movies they watched from all concepts
  usersGroup=pd.DataFrame(part_sum_for_group_concept.loc[argmax_user])
  usersGroup.columns = [f"percentage of {group_concept} movies"]

  return usersGroup

# MF help functions

In [ ]:
def rec_gen(movie_num, user, df_b_u_emb, df_b_i_emb, P, Q):

  PQ_user = expit(df_b_u_emb.iloc[user,0] + pd.concat([df_b_i_emb, P.dot(Q.T)], axis=1).sum(axis=1))
  top_rec_user = PQ_user.nlargest(movie_num)
  top_rec_user_id = top_rec_user.index.tolist()

  return top_rec_user_id

In [ ]:
def normalize_val(recommend_tuple_list):
  normalized_data = [(item[0], (item[1] - min(recommend_tuple_list, key=lambda
                      x: x[1])[1]) / (max(recommend_tuple_list, key=lambda x:
                    x[1])[1] - min(recommend_tuple_list, key=lambda x: x[1])[1])) for item in recommend_tuple_list]
  return normalized_data

In [ ]:
def normalize_matrix(matrix):
    matrix = matrix.float()
    min_val,_ = matrix.min(axis=0)
    max_val,_ = matrix.max(axis=0)
    normalized_matrix = (matrix - min_val) / (max_val - min_val)
    return np.nan_to_num(normalized_matrix)

#Converts sparse input to dense tensor.
def convert_to_dense_tensor(sparse_matrix):
    dense_matrix = np.array(sparse_matrix)
    dense_tensor = torch.tensor(dense_matrix, dtype=torch.float32)
    return dense_tensor

# Ensures uniform input dimensions.
def pad_or_truncate_tensor(tensor1, target_dim):
    flattened_tensor = tensor1.reshape(tensor1.shape[0], -1)
    if flattened_tensor.shape[1] < target_dim:
        padded_tensor = torch.nn.functional.pad(flattened_tensor, (0, target_dim - flattened_tensor.size(1)))
    else:
        padded_tensor = flattened_tensor[:, :target_dim]
    return padded_tensor

# Ensures uniform input dimensions.
def pad_or_truncate_tensor_0(tensor1, target_dim):
    flattened_tensor = tensor1.reshape(tensor1.shape[0], -1)
    if flattened_tensor.shape[0] < target_dim:
        padded_tensor = torch.nn.functional.pad(flattened_tensor, (0, target_dim - flattened_tensor.size(1)))
    else:
        padded_tensor = flattened_tensor[:, :target_dim]
    return padded_tensor

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)


# Lists Correlation Metrics

In [ ]:
def rbo(list1, list2, p=0.9):
    """
    Calculate Rank Biased Overlap (RBO) between two ranked lists.

    Args:
        list1 (list): The first ranked list.
        list2 (list): The second ranked list.
        p (float): The probability of considering ranks deeper in the list (default=0.9).
                   Higher values give more weight to deeper ranks.

    Returns:
        float: The RBO score between the two lists.
    """
    # Lengths of the two lists
    len1, len2 = len(list1), len(list2)
    max_depth = max(len1, len2)

    # Track cumulative overlap
    cumulative_overlap = 0
    agreement = 0  # Overlap count at each depth

    for d in range(1, max_depth + 1):
        # Get the top-d elements from both lists
        top_d1 = set(list1[:d])
        top_d2 = set(list2[:d])

        # Calculate overlap at depth d
        agreement = len(top_d1.intersection(top_d2))

        # Weighted contribution to RBO
        cumulative_overlap += (p ** (d - 1)) * (agreement / d)

    # RBO formula
    rbo_score = (1 - p) * cumulative_overlap
    return rbo_score




In [ ]:
from scipy.stats import kendalltau

def kendall_tau(list1, list2):
    """
    Calculate Kendall Tau correlation between two ranked lists.

    Args:
        list1 (list): The first ranked list.
        list2 (list): The second ranked list.

    Returns:
        float: Kendall Tau correlation coefficient.
    """
    # Create ranking dictionaries for both lists
    rank1 = {item: rank for rank, item in enumerate(list1, 1)}
    rank2 = {item: rank for rank, item in enumerate(list2, 1)}

    # Make a union of all elements
    all_items = list(set(list1) | set(list2))

    # Convert ranks to aligned lists (fill missing elements with default ranks)
    aligned_rank1 = [rank1.get(item, len(list1) + 1) for item in all_items]
    aligned_rank2 = [rank2.get(item, len(list2) + 1) for item in all_items]

    # Use scipy's kendalltau for calculation
    tau, _ = kendalltau(aligned_rank1, aligned_rank2)
    return tau

# Monosemanticity Score

In [ ]:
def ms_score_new(df_cosine_sim_matrix, latents_items):
  A = latents_items.detach().cpu().numpy()
  scaler = MinMaxScaler()
  A_norm = scaler.fit_transform(A)

  N = A.shape[0]
  K=30 

  MS_scores_topK = {}

  for k in range(latents_items.shape[1]):  # for each neuron
      if k in range(latents_items.shape[1]):
        a_k = A_norm[:, k]

        top_k_idx = np.argsort(a_k)[-K:]
        top_k_vals = a_k[top_k_idx]

        # outer productof activation matrix
        R_k = np.outer(top_k_vals, top_k_vals)
        np.fill_diagonal(R_k, 0)

        # similarity matrix
        df_cosine_sim_matrix_array = df_cosine_sim_matrix.values
        S_top_k = df_cosine_sim_matrix_array[np.ix_(top_k_idx, top_k_idx)]
        np.fill_diagonal(S_top_k, 0)

        # Calculate MS
        denom = np.sum(R_k)
        MS_k = np.sum(R_k * S_top_k) / denom if denom != 0 else 0
        MS_scores_topK[k] = MS_k

  return sum(MS_scores_topK.values())/latents_items.shape[1]

In [ ]:
def ms_score_all_neurons(df_cosine_sim_matrix, latents_items):
  A = latents_items.detach().cpu().numpy()
  scaler = MinMaxScaler()
  A_norm = scaler.fit_transform(A)

  N = A.shape[0]
  K=30 

  MS_scores_topK = {}

  for k in range(latents_items.shape[1]):  # for each neuron
      if k in range(latents_items.shape[1]):
        a_k = A_norm[:, k]

        top_k_idx = np.argsort(a_k)[-K:]
        top_k_vals = a_k[top_k_idx]

        # outer productof activation matrix
        R_k = np.outer(top_k_vals, top_k_vals)
        np.fill_diagonal(R_k, 0)

        # similarity matrix
        df_cosine_sim_matrix_array = df_cosine_sim_matrix.values
        S_top_k = df_cosine_sim_matrix_array[np.ix_(top_k_idx, top_k_idx)]
        np.fill_diagonal(S_top_k, 0)

        # Calculate MS
        denom = np.sum(R_k)
        MS_k = np.sum(R_k * S_top_k) / denom if denom != 0 else 0
        MS_scores_topK[k] = MS_k

  return MS_scores_topK.values()

# Recommendations Extraction- NCF

In [ ]:
def get_top_k_recommendations(model, user_id, candidate_item_ids, K):
    """
    Returns the top K recommended item IDs for a given user.

    Args:
        model: Trained NeuralCollaborativeFiltering model.
        user_id: The user ID for which recommendations are needed.
        candidate_item_ids: List (or tensor) of candidate item IDs.
        K: The number of top recommendations to return.

    Returns:
        A list of the top K recommended item IDs.
    """
    # Create a tensor for the user repeated for each candidate item.
    user_tensor = torch.tensor([user_id] * len(candidate_item_ids), dtype=torch.long)
    item_tensor = torch.tensor(candidate_item_ids, dtype=torch.long)

    # Set the model to evaluation mode and disable gradient computation.
    model.eval()
    with torch.no_grad():
        # Get predictions for all candidate items.
        scores  = model(user_tensor, item_tensor)

    sorted_indices = torch.argsort(scores, descending=True)

    # Select the top K item IDs.
    top_k_indices = sorted_indices[:K]
    top_k_item_ids = [candidate_item_ids[i] for i in top_k_indices]
    bottom_k_indices = sorted_indices[-K:-1]
    top_k_item_ids = [candidate_item_ids[i] for i in top_k_indices]
    recommendations = [(ratings_matrix.columns[top_k_item_ids[i]], round(float(scores[top_k_item_ids[i]].data),6)) for i in range(K)]

    return scores, top_k_item_ids,recommendations

In [ ]:
def get_top_k_recommendations_flex(model, user_id, candidate_item_ids, K):
    """

    TopK recommendation list for reconstructed users' and items'
    latent representations.

    Returns the top K recommended item IDs for a given user.

    Args:
        model: Trained NeuralCollaborativeFiltering model.
        user_id: The user ID for which recommendations are needed.
        candidate_item_ids: List (or tensor) of candidate item IDs.
        K: The number of top recommendations to return.

    Returns:
        A list of the top K recommended item IDs.
    """
    # Create a tensor for the user repeated for each candidate item.
    user_tensor = torch.tensor([user_id] * len(candidate_item_ids), dtype=torch.long)
    item_tensor = torch.tensor(candidate_item_ids, dtype=torch.long)

    # Set the model to evaluation mode and disable gradient computation.
    model.eval()
    with torch.no_grad():

        #-----
        user_emb = model.user_embedding(user_tensor).detach().clone()
        item_emb = model.item_embedding(item_tensor).detach().clone()
        user_rec, user_encoded = sae_model(user_emb)
        item_rec, item_encoded = sae_model(item_emb)
        #------

        x_hat = torch.cat([user_rec, item_rec], dim=-1)
        y_hat = model.fc_layers(x_hat)  # (batch, 1)
        scores = y_hat.squeeze().detach().clone()

    # Sort candidate items based on their scores (descending order).
    # torch.argsort returns indices that would sort the tensor.
    sorted_indices = torch.argsort(scores, descending=True)

    # Select the top K item IDs.
    top_k_indices = sorted_indices[:K]
    top_k_item_ids = [candidate_item_ids[i] for i in top_k_indices]
    bottom_k_indices = sorted_indices[-K:-1]
    top_k_item_ids = [candidate_item_ids[i] for i in top_k_indices]
    recommendations = [(ratings_matrix.columns[top_k_item_ids[i]], round(float(scores[top_k_item_ids[i]].data),6)) for i in range(K)]
    return scores, top_k_item_ids,recommendations



#---------------------------------------------------------------------


def get_top_k_recommendations_flex_item(model, user_id, candidate_item_ids, K,item_rec_replace, item_id_replace):
    """
    TopK recommendation list for reconstructed modified item's latent
    representation & reconstructed user's latent representation.

    Args:
        model: Trained NeuralCollaborativeFiltering model.
        user_id: The user ID for which recommendations are needed.
        candidate_item_ids: List (or tensor) of candidate item IDs.
        K: The number of top recommendations to return.

    Returns:
        A list of the top K recommended item IDs.
    """
    # Create a tensor for the user repeated for each candidate item.
    user_tensor = torch.tensor([user_id] * len(candidate_item_ids), dtype=torch.long)
    item_tensor = torch.tensor(candidate_item_ids, dtype=torch.long)

    # Set the model to evaluation mode and disable gradient computation.
    model.eval()
    sae_model.eval()
    with torch.no_grad():
        # Get predictions for all candidate items.
        #-----
        user_emb = model.user_embedding(user_tensor).detach().clone()
        item_emb = model.item_embedding(item_tensor).detach().clone()
        user_rec, user_encoded = sae_model(user_emb)
        item_rec, item_encoded = sae_model(item_emb)
        #------
        item_rec_replace = item_rec_replace.unsqueeze(-1)
        item_rec[item_id_replace,:] = item_rec_replace.t()

        x_hat = torch.cat([user_rec, item_rec], dim=-1)
        y_hat = model.fc_layers(x_hat)  # (batch, 1)
        scores = y_hat.squeeze().detach().clone()

    # Sort candidate items based on their scores (descending order).
    # torch.argsort returns indices that would sort the tensor.
    sorted_indices = torch.argsort(scores, descending=True)

    # Select the top K item IDs.
    top_k_indices = sorted_indices[:K]
    top_k_item_ids = [candidate_item_ids[i] for i in top_k_indices]
    bottom_k_indices = sorted_indices[-K:-1]
    top_k_item_ids = [candidate_item_ids[i] for i in top_k_indices]
    recommendations = [(ratings_matrix.columns[top_k_item_ids[i]], round(float(scores[top_k_item_ids[i]].data),6)) for i in range(K)]
    return scores, top_k_item_ids,recommendations



#---------------------------------------------------------------------

def get_top_k_recommendations_flex_user(model, user_id, candidate_item_ids, K,user_latent_replace):
    """
    TopK recommendation list for reconstructed modified users's latent
    representation & reconstructed items's latent representation.

    Args:
        model: Trained NeuralCollaborativeFiltering model.
        user_id: The user ID for which recommendations are needed.
        candidate_item_ids: List (or tensor) of candidate item IDs.
        K: The number of top recommendations to return.

    Returns:
        A list of the top K recommended item IDs.
    """
    # Create a tensor for the user repeated for each candidate item.
    user_tensor = torch.tensor([user_id] * len(candidate_item_ids), dtype=torch.long)
    item_tensor = torch.tensor(candidate_item_ids, dtype=torch.long)

    # Set the model to evaluation mode and disable gradient computation.
    model.eval()
    sae_model.eval()
    with torch.no_grad():
        # Get predictions for all candidate items.

        #-----
        user_emb_modificate = user_latent_replace.repeat(3706, 1)
        item_emb = model.item_embedding(item_tensor).detach().clone()
        user_rec, user_encoded = sae_model(user_emb_modificate)
        item_rec, item_encoded = sae_model(item_emb)
        #-----

        x_hat = torch.cat([user_rec, item_rec], dim=-1)
        y_hat = model.fc_layers(x_hat)  # (batch, 1)
        scores = y_hat.squeeze().detach().clone()

    # Sort candidate items based on their scores (descending order).
    # torch.argsort returns indices that would sort the tensor.
    sorted_indices = torch.argsort(scores, descending=True)

    # Select the top K item IDs.
    top_k_indices = sorted_indices[:K]
    top_k_item_ids = [candidate_item_ids[i] for i in top_k_indices]
    bottom_k_indices = sorted_indices[-K:-1]
    top_k_item_ids = [candidate_item_ids[i] for i in top_k_indices]
    recommendations = [(ratings_matrix.columns[top_k_item_ids[i]], round(float(scores[top_k_item_ids[i]].data),6)) for i in range(K)]
    return scores, top_k_item_ids,recommendations

## Evaluation metrics:

NDCG

In [ ]:
def ndcg_calc(k, model_type = 'MF', pos_idx_ex_hidden, model=model):

  ndcg_sum = 0
  ndcg_sum_all=[]
  total = 0

  for user_id in range(len(pos_idx_ex_hidden)):
      hidden_items = pos_idx_ex_hidden[user_id]  # The relevant (hidden) items for the user
      if model_type == 'MF':
        recommendations = mf_recommender.recommend(user_id, k)
      else:
        recommendations = get_top_k_recommendations(model, user_id, list(range(3706)), k)[2]

      # Compute DCG
      dcg = 0
      for rank, (item_id, score) in enumerate(recommendations):
          if item_id in hidden_items:
            # rank is the place of the recoommendation
              dcg += score / np.log2(rank + 1 + 1)  # rank + 2 because rank starts from 0

      # Compute IDCG (Ideal DCG)
      idcg = sum(1 / np.log2(i + 2) for i in range(min(len(hidden_items), k)))

      ndcg = dcg / idcg if idcg > 0 else 0
      ndcg_sum += ndcg
      ndcg_sum_all.append(ndcg)
      total += 1

  return avg_NDCG_at_k, ndcg_sum_all

MRR at k=20

In [ ]:
def mmr_calc(k, model_type = 'MF', pos_idx_ex_hidden, model=model):

  rr_sum = 0
  total = 0

  for user_id in range(len(pos_idx_ex_hidden)):
    hidden_items = pos_idx_ex_hidden[user_id]
    if model_type == 'MF':
        recommendations = mf_recommender.recommend(user_id, k)
    else:
        recommendations = get_top_k_recommendations(model, user_id, list(range(3706)), k)[2]
    used_flag = 0
    for item_id in hidden_items:
        for rank, (rec_item_id, _) in enumerate(recommendations):
            if rec_item_id == item_id and used_flag==0:
                used_flag = 1
                rr_sum += 1 / (rank + 1)  # add 1 since the counting starts
                                          # here from 0
    total += 1
  mrr_at_k = rr_sum / len(pos_idx_ex_hidden)

  return mrr_at_k

Hit rate at k

In [ ]:
def hit_rate_calc(k, model_type = 'MF', pos_idx_ex_hidden, model=model):

  hit_rate_at_k = 0
  num_user_w_rel_item = 0

  for user_id in range(len(pos_idx_ex_hidden)):
    hidden_items = pos_idx_ex_hidden[user_id]
    if model_type == 'MF':
        recommendations = mf_recommender.recommend(user_id, k)
    else:
        recommendations = get_top_k_recommendations(model, user_id, list(range(3706)), k)[2]
    used_flag = 0
    for item_id in hidden_items:
        for rank, (rec_item_id, _) in enumerate(recommendations):
            if rec_item_id == item_id and used_flag==0:
                used_flag = 1
                num_user_w_rel_item += 1
  hit_rate_at_k = num_user_w_rel_item/len(pos_idx_ex_hidden)

  return hit_rate_at_k

Mean percentile rank

In [ ]:
def mpr_calc(model_type = 'MF', pos_idx_ex_hidden, model=model):

  mean_percentile_rank = 0
  percentile_rank = 0

  for user_id in range(len(pos_idx_ex_hidden)):
    hidden_items = pos_idx_ex_hidden[user_id]
    if model_type == 'MF':
        recommendations = mf_recommender.recommend(user_id, 3706)
    else:
        recommendations = get_top_k_recommendations(model, user_id, list(range(3706)), 3706)[2]

    rr = 0
    for item_id in hidden_items:
        for rank, (rec_item_id, _) in enumerate(recommendations):
            if rec_item_id == item_id:
                rr += (rank/3706)*100
    percentile_rank += rr/len(hidden_items)
  mean_percentile_rank = percentile_rank/len(pos_idx_ex_hidden)

  return mean_percentile_rank